In [ ]:
import csv
import json
import pathlib
from collections import defaultdict
csv.field_size_limit(sys.maxsize)

In [ ]:
delete_studies = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/11/deletion_imaging_study_ACR_20211115.tsv"

to_delete_studies = []

with open(delete_studies) as deletion:
    tsv_reader = csv.DictReader(deletion, delimiter="\t")
    for record in tsv_reader:
        study_id = record["submitter_id"]
        to_delete_studies.append(study_id)

In [ ]:
delete_studies = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/11/RemoveHeadsFrom20211004.txt"

with open(delete_studies) as deletion:
    tsv_reader = csv.DictReader(deletion, delimiter="\t")
    for record in tsv_reader:
        study_id = record["submitter_id"].split("_")[1]
        to_delete_studies.append(study_id)

In [ ]:
all_packages = []
with open("all_packages.tsv") as all_packages_file:
    tsv_reader = csv.DictReader(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE)
    for record in tsv_reader:
        filename = record["file_name"].split("/")[0]
        if filename in to_delete_studies:
            print(filename)
            continue
        all_packages.append(record)

In [ ]:
with open("all_packages_submit.tsv", "w") as all_packages_file:
    fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'url', 'file_name', 'package_contents']
    tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    tsv_writer.writeheader()

    for package in all_packages:
        tsv_writer.writerow(package)

In [ ]:
sequestration_filename = "/Users/andrew/CTDS/midrc-etl/v2/sequestration/ACR_November_imaging_studies_split/imaging_study_sequestration_location.tsv"

datasets = {}

with open(sequestration_filename) as sequestration_file:
    tsv_reader = csv.DictReader(sequestration_file, delimiter="\t")

    for record in tsv_reader:
        datasets[record["submitter_id"]] = record["dataset"]

In [ ]:
packages_filename = "/Users/andrew/CTDS/midrc-etl/v2/all_packages_submit.tsv"

with open(packages_filename) as packages_file:
    tsv_reader = csv.DictReader(packages_file, delimiter="\t")

    with open("all_packages_seq.tsv", "w") as all_packages_file:
        fieldnames = ['record_type', 'dataset', 'guid', 'md5', 'size', 'authz', 'url', 'file_name', 'package_contents']
        tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
        tsv_writer.writeheader()

        for record in tsv_reader:
            study_id = record["file_name"].split("/")[0]

            r = record
            r["dataset"] = datasets[study_id]

            tsv_writer.writerow(r)

In [ ]:
packages_filename = "/Users/andrew/CTDS/midrc-etl/v2/all_packages_submit.tsv"

with open(packages_filename) as packages_file:
    tsv_reader = csv.DictReader(packages_file, delimiter="\t")

    with open("to_index_OPEN_imaging_study_ACR_20211115.tsv.tsv", "w") as all_packages_file:
        fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'url', 'file_name', 'package_contents']
        tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
        tsv_writer.writeheader()

        for record in tsv_reader:
            study_id = record["file_name"].split("/")[0]

            if datasets[study_id] == "Open":
                tsv_writer.writerow(record)

In [ ]:
packages_filename = "/Users/andrew/CTDS/midrc-etl/v2/all_packages_submit.tsv"

with open(packages_filename) as packages_file:
    tsv_reader = csv.DictReader(packages_file, delimiter="\t")

    with open("to_index_SEQ_imaging_study_ACR_20211115.tsv.tsv", "w") as all_packages_file:
        fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'url', 'file_name', 'package_contents']
        tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
        tsv_writer.writeheader()

        for record in tsv_reader:
            study_id = record["file_name"].split("/")[0]

            if datasets[study_id] == "Seq":
                tsv_writer.writerow(record)

In [ ]:
with open("indexed_OPEN_imaging_study_ACR_20211115.tsv") as f:
    tsv_reader = csv.DictReader(f, delimiter="\t")
    with open("to_index_OPEN_imaging_study_ACR_20211115.tsv.tsv", "w") as all_packages_file:
        fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'urls', 'file_name', 'package_contents']
        tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
        tsv_writer.writeheader()

        for record in tsv_reader:
            record["authz"] = '["/programs/Open/projects/A1"]'
            tsv_writer.writerow(record)

In [ ]:
with open("indexed_SEQ_imaging_study_ACR_20211115.tsv") as f:
    tsv_reader = csv.DictReader(f, delimiter="\t")
    with open("to_index_SEQ_imaging_study_ACR_20211115.tsv", "w") as all_packages_file:
        fieldnames = ['record_type', 'guid', 'md5', 'size', 'authz', 'urls', 'file_name', 'package_contents']
        tsv_writer = csv.DictWriter(all_packages_file, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
        tsv_writer.writeheader()

        for record in tsv_reader:
            record["authz"] = '["/programs/SEQ_Open/projects/A3"]'
            tsv_writer.writerow(record)